# Cryptocurrency Implementation Project

**Author:** Sehong Park  

# Part 1: Design Discussion

In this section, we address the core architectural decisions of the cryptocurrency design based on the following requirements:

1. a unit of currency.
2. the details of any transaction in terms of what needs to be recorded. 
3. a method for certifying ownership and authorizing transactions.  Basically this involves a signing algorithms.
4. how you will avoid the "double spending" problem by setting up a mechanism for users to verify a unit of currency is actually available.  Blockchain is the mechanism Bitcoin uses.t
5. how currency is to be generated, typically this would be in terms of users offering some sort of service.  For example the "mining" of Bitcoins is done by verifying transactions on its Blockchain.
6. any other issues specific to your design.

## 1. Unit of Currency: Dynamic Precision Model

To avoid fixing a permanent minimum unit of currency, we allow divisibility to
increase as the network grows. If the currency becomes more valuable, users
naturally need smaller units. Instead of performing a redenomination, we tie
the number of allowed decimal places to the block difficulty.

**Rule:**  
Allowed decimal places = difficulty of the latest block.

Examples:  
- difficulty = 0 → 50  
- difficulty = 2 → 50.00  
- difficulty = 5 → 50.00000  

Thus the currency automatically becomes more divisible as mining difficulty rises.

**Precision Check:**  
The validation logic ensures that a transaction amount does not exceed the
allowed decimal places. This check is implemented in the  
`TransactionValidator` class:

```python
def check_precision(self, amount: Decimal, difficulty: int) -> bool:
    s = str(amount)
    if '.' not in s:
        return True
    decimals = len(s.split('.')[1])
    return decimals <= difficulty

## 2. Transaction Data Details & Structure

In this project, we implement two types of transactions. Each field serves a critical role in ensuring security, consensus, and state management.

#### A. Account Transaction (`AccountTx`)
Standard peer-to-peer cryptocurrency transfer used for value exchange between users.

* **sender** (`str`): The wallet address of the sender.
    * *Why needed:* Identifies which account balance should be debited in the Global State.
* **recipient** (`str`): The wallet address of the receiver.
    * *Why needed:* Identifies which account balance should be credited.
* **amount** (`Decimal`): The quantity of CryptoCoin to transfer.
    * *Why needed:* Represents the actual economic value moving between accounts. This is validated against the sender's current balance.
* **fee** (`Decimal`): The transaction fee paid to the miner.
    * *Why needed:* Incentivizes miners to prioritize this transaction (Fee Market) and acts as an economic cost barrier to prevent spam (DoS) attacks.
* **nonce** (`int`): A strictly increasing counter specific to the sender's account.
    * *Why needed:* Crucial for preventing **Replay Attacks**. It ensures that a signed transaction cannot be intercepted and broadcasted multiple times to drain funds.
* **timestamp** (`float`): The creation time of the transaction.
    * *Why needed:* Ensures temporal integrity by preventing transactions from being broadcasted with timestamps too far in the future or past (Drift Check).
* **signature** (`hex`): The Ed25519 digital signature of the transaction data.
    * *Why needed:* Cryptographically proves that the `sender` actually authorized this transaction and ensures the data has not been tampered with (Integrity).
* **pubkey** (`hex`): The raw public key of the sender.
    * *Why needed:* Since the `sender` address is a hash, the raw public key must be revealed to verify the `signature`.

#### B. Coinbase Transaction (`CoinbaseTx`)
A special transaction generated by the system to reward miners.

* **recipient** (`str`): The address of the miner who solved the PoW puzzle.
    * *Why needed:* Designates the beneficiary of the block reward and accumulated transaction fees.
* **amount** (`Decimal`): The sum of the Block Reward (e.g., 50 CC) and total fees from transactions in the block.
    * *Why needed:* This is the sole mechanism for **Minting** (creating new currency supply) and provides the economic incentive for securing the network.

## 3. Certification Method: Ed25519 Signatures

To secure user funds and authorize state transitions without a central authority, this system utilizes **Public Key Cryptography**. We specifically employ the **Ed25519** signature scheme.

#### A. Signing Algorithm: Ed25519
Instead of the traditional ECDSA used by Bitcoin (secp256k1), we use the **Ed25519** (Edwards-curve Digital Signature Algorithm).
* **Why Ed25519?**:
    * **Performance:** It offers faster signing and verification speeds compared to ECDSA.
    * **Security:** It is designed to be immune to side-channel attacks that plague standard ECDSA implementations.
    * **Compactness:** It uses small 32-byte public keys, reducing storage requirements on the blockchain.

#### B. Certifying Ownership (Wallet & Address)
Ownership of funds is defined by the possession of a **Private Key**. The system does not store "accounts" like a bank; it tracks balances assigned to "Addresses."
* **Private Key (`sk`)**: A randomly generated secret that must be kept secure. It allows the user to spend funds.
* **Public Key (`vk`)**: Derived mathematically from the private key. It is shared with the network to verify signatures.
* **Address Generation**: The user's address is a hashed version of the Public Key.
    * `Address = Base58Check( RIPEMD160( SHA256( Public Key ) ) )`
    * **Why:** Hashing provides an extra layer of security (quantum resistance until the first spend) and creates a shorter, human-readable format for receiving funds.

#### C. Authorizing Transactions (Digital Signatures & Nonce)
When a user wants to transfer funds, they must prove they own the address without revealing their private key.

1.  **Signing (Sender side):**
    * The user creates a transaction message including the **current Nonce** (e.g., 0).
    * The `Wallet` class uses the **Private Key** to generate a cryptographic **Signature** of this message.

2.  **Verification (Network side):**
    * Nodes check two conditions before accepting the transaction:
        1.  **Identity:** Does `Hash(Public Key)` match the `Sender Address`?
        2.  **Integrity:** Is the `Signature` valid for this message using the `Public Key`?

3.  **Replay Protection (Nonce Check):**
    * Even if the signature is valid, the system compares the transaction's `nonce` with the sender's current state in the ledger.
    * **Rule:** `Tx Nonce` must equal `Account Nonce`.
    * **Update:** If valid, the account's nonce is incremented by 1 (`Nonce += 1`).
    * **Why:** This strict ordering prevents attackers from re-broadcasting an old, already-signed transaction to spend funds multiple times (Replay Attack).

## 4. Double Spending Prevention Mechanism

Double spending is the risk that a digital currency holder could spend the same unit of currency more than once. Since digital tokens are just data, they can be easily duplicated. To prevent this, our system employs a multi-layered verification mechanism centered around the **Blockchain**.

####  Verification of Available Funds (State Check)
The primary mechanism to verify if a unit of currency is actually available is the **Global State Ledger**.
* **Mechanism:** Unlike physical cash, our system maintains a `StateManager` that tracks the exact balance of every address.
* **The Check:** Before any transaction is accepted, the `TransactionValidator` performs a strict check: `Sender Balance >= Amount + Fee`.
* **Result:** If a user attempts to spend funds they have already used in a previous transaction (which reduced their recorded balance), this check fails, and the transaction is rejected immediately.

####  Ordering via Nonces
To prevent a user from broadcasting two conflicting transactions simultaneously (e.g., spending the same 50 coins to Bob and Charlie at the same time):
* **Mechanism:** Every account has a strictly increasing counter called a `nonce`.
* **The Check:** The system enforces that `Tx.Nonce == Account.Current_Nonce`.
* **Result:** Once the first transaction is accepted, the account's nonce increments. Any subsequent attempt to re-use that nonce for a double-spend transaction will be rejected as invalid.

####  Global Consensus (Proof-of-Work)
In a distributed network, different nodes might see conflicting transactions at different times. The **Blockchain** serves as the final arbiter.

* **Mechanism:** Miners solve a cryptographic puzzle (Proof-of-Work) to seal transactions into a block.
* **Resolution:** If two conflicting transactions make it into competing blocks (a fork), the network adopts the **Longest Chain Rule**. Only the transaction in the longest chain is considered valid history; the other is discarded, ensuring the currency is spent only once.

## 5. Currency Generation: Mining & Coinbase

In this system, currency is not issued by a central bank but is generated algorithmically through a process called **Mining**. This aligns with the principle that currency should be earned by providing a valuable service to the network.

#### 1. The Service: Security & Validation
Miners provide the critical service of maintaining the ledger's integrity.
* **Validation:** They collect pending transactions from the Mempool, verify their digital signatures, and ensure no double-spending occurs.
* **Security (Proof-of-Work):** They expend computational energy (CPU power) to solve a cryptographic puzzle (finding a hash below a specific target). This work secures the blockchain history against tampering, as rewriting the history would require re-doing all the work.

#### 2. The Reward: Coinbase Transaction
As compensation for this service, the protocol allows the miner to include a special transaction called the **Coinbase Transaction** at the beginning of the block.
* **Minting:** This transaction has no sender (input) and creates new coins out of thin air.
* **Components:**
    * **Block Reward:** A fixed amount (e.g., 50 CC) minted per block.
    * **Transaction Fees:** The sum of fees from all transactions included in the block.
    
Consequently, **Mining** is the sole mechanism for introducing new currency into the circulation supply.

## 6. Design-Specific Issues & Limitations

This prototype demonstrates the core mechanics of a blockchain, but specific design choices and the nature of a "Toy Model" introduce certain limitations.

#### 1. Sequential Transaction Processing (Strict Ordering)
Our implementation enforces a strict rule where `Tx.Nonce` must exactly match the `Account.Nonce` stored in the committed ledger state.
* **The Constraint:** Transactions must be processed strictly in order (Nonce 0, 1, 2...).
* **Impact:** This creates a concurrency bottleneck known as **Head-of-Line Blocking**. If a user broadcasts `Tx #0` (Low Fee) and `Tx #1` (High Fee), the network cannot process `Tx #1` until `Tx #0` is confirmed, regardless of the higher fee.

#### 2. Data Persistence (Volatile Memory)
Currently, the `StateManager` stores all account balances and nonces in Python dictionaries (RAM).
* **The Issue:** The system lacks a persistent storage layer (like LevelDB or RocksDB).
* **Impact:** If the node software terminates or crashes, the entire ledger state is lost. Furthermore, as the blockchain grows, the memory requirement scales linearly, eventually exceeding available RAM.

#### 3. Network Layer Abstraction
The simulation models P2P networking via direct method calls between objects in a single process (e.g., `node_B.receive_chain(node_A)`).
* **The Issue:** It lacks a true network stack (TCP/IP, Socket).
* **Impact:** Real-world challenges such as network latency, packet loss, and peer discovery are not simulated, which are critical factors in actual consensus convergence.

#### 4. Mining Algorithm Efficiency
The Proof-of-Work loop is implemented in pure Python.
* **The Issue:** Python's Global Interpreter Lock (GIL) and interpreted nature make the mining process significantly slower than C++ or hardware (ASIC) implementations.
* **Impact:** This makes the current prototype unsuitable for high-difficulty environments, serving primarily as an educational demonstration of the consensus logic.

# Part 2: Technical Implementation

This section translates the theoretical design discussed above into a functional Python prototype. The implementation covers the wallet logic, transaction models, consensus engine (PoW), and network simulation.

## 1. Libraries and Setup

This project utilizes standard cryptographic libraries to ensure security and integrity.

* **Hashing:** Uses `hashlib` to implement **SHA-256** and **RIPEMD-160** algorithms for data integrity and address generation.
* **Digital Signatures:** Adopts **Ed25519** instead of the traditional ECDSA (secp256k1).
    * **Reasoning:** Ed25519 offers faster verification speeds and better resistance against side-channel attacks compared to RSA or ECDSA. It also utilizes compact 32-byte public keys, optimizing blockchain storage.

In [24]:
import hashlib
import time
import json
import copy
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any, Union, Callable
from decimal import Decimal

# External Libraries
import base58
try:
    from cryptography.hazmat.primitives.asymmetric import ed25519
    from cryptography.hazmat.primitives import serialization
except ImportError:
    raise ImportError("cryptography library required (pip install cryptography)")

## 2. Constants and Helper Functions

These helper functions establish the data structures required for blockchain integrity.

* **JSON Hashing:** Serializes objects to generate SHA-256 hashes, ensuring consistent data representation.
* **Merkle Tree:** efficient structure to verify the integrity of transactions within a block.
    * It recursively hashes pairs of transaction hashes until a single **Merkle Root** remains.
    * **Benefit:** The Merkle Root is included in the block header, allowing light clients to verify the inclusion of specific transactions without downloading the entire block body.

In [25]:
# --- Constants ---
ADDRESS_VERSION_BYTE = b'\\x00'
MAX_FUTURE_DRIFT = 2 * 3600  # 2 hours

# --- Helper Functions ---
def hash_json(obj: Any, hashfunc: Callable = hashlib.sha256) -> str:
    def decimal_default(o):
        if isinstance(o, Decimal): return str(o)
        raise TypeError
    data = json.dumps(obj, separators=(',', ':'), sort_keys=True, default=decimal_default).encode()
    return hashfunc(data).hexdigest()

def merkle_root(hashes: List[str], hashfunc: Callable = hashlib.sha256) -> str:
    if not hashes: return hashfunc(b"").hexdigest()
    layer = hashes[:]
    while len(layer) > 1:
        if len(layer) % 2 == 1: layer.append(layer[-1])
        nxt = []
        for i in range(0, len(layer), 2):
            concat = hashfunc((layer[i] + layer[i+1]).encode()).hexdigest()
            nxt.append(concat)
        layer = nxt
    return layer[0]

## 3. Wallet and Identity Management (Ed25519)

This class handles user identity and key management.

* **Key Generation:** Generates a private/public key pair using the Ed25519 algorithm.
* **Address Generation:** Derives a human-readable address from the public key using a multi-step hashing process:
    1.  **SHA-256**: Hashes the public key.
    2.  **RIPEMD-160**: Compresses the hash for shorter addresses.
    3.  **Base58Check Encoding**: Adds a checksum to prevent transcription errors.
* **Signature Verification:** Verifies transaction signatures using the sender's public key to ensure authenticity.

In [26]:
class Wallet:
    def __init__(self):
        self.sk = ed25519.Ed25519PrivateKey.generate()
        self.vk = self.sk.public_key()
        self._pub_bytes = self.vk.public_bytes(
            encoding=serialization.Encoding.Raw,
            format=serialization.PublicFormat.Raw
        )
        self.address = self._generate_address(self._pub_bytes)
    
    @staticmethod
    def _generate_address(pub_bytes: bytes) -> str:
        sha = hashlib.sha256(pub_bytes).digest()
        hash160 = hashlib.new('ripemd160', sha).digest()
        payload = ADDRESS_VERSION_BYTE + hash160
        checksum = hashlib.sha256(hashlib.sha256(payload).digest()).digest()[0:4]
        return base58.b58encode(payload + checksum).decode()

    def get_public_key_hex(self) -> str:
        return self._pub_bytes.hex()
        
    def sign_message(self, message: bytes) -> bytes:
        return self.sk.sign(message)
    
    @staticmethod
    def verify_signature(signature: bytes, message: bytes, public_key_hex: str) -> bool:
        try:
            pub_bytes = bytes.fromhex(public_key_hex)
            loaded_pub_key = ed25519.Ed25519PublicKey.from_public_bytes(pub_bytes)
            loaded_pub_key.verify(signature, message)
            return True
        except Exception:
            return False

    @staticmethod
    def address_from_pubkey_hex(public_key_hex: str) -> str:
        return Wallet._generate_address(bytes.fromhex(public_key_hex))

## 4. Transaction Models

The protocol defines two distinct transaction types:

* **AccountTx (Peer-to-Peer):** Handles asset transfers between users. Unlike Bitcoin's UTXO model, this uses an **Account-based Model** (similar to Ethereum).
    * **Nonce:** A strictly increasing number to prevent Replay Attacks.
    * **Fee:** An economic incentive for miners to prioritize the transaction.
* **CoinbaseTx (Minting):** A special system transaction created by miners to claim block rewards. It has no sender and serves as the source of new currency supply.

In [27]:
@dataclass
class AccountTx:
    sender: str
    recipient: str
    amount: Decimal
    fee: Decimal
    nonce: int = 0
    pubkey: Optional[str] = None
    signature: Optional[str] = None
    timestamp: float = field(default_factory=time.time)

    def message(self) -> dict:
        return {
            "type": "account", "sender": self.sender, "recipient": self.recipient,
            "amount": str(self.amount), "fee": str(self.fee), "nonce": self.nonce,
            "timestamp": int(self.timestamp)
        }
    
    def txhash(self, hashfunc: Callable = hashlib.sha256) -> str:
        return hash_json({**self.message(), "pubkey": self.pubkey, "signature": self.signature}, hashfunc)
    
    def sign(self, wallet):
        if wallet.address != self.sender: raise ValueError("Address mismatch")
        self.pubkey = wallet.get_public_key_hex()
        msg_bytes = json.dumps(self.message(), separators=(',', ':'), sort_keys=True).encode()
        self.signature = wallet.sign_message(msg_bytes).hex()
        
    def verify_signature(self) -> bool:
        if not self.pubkey or not self.signature: return False
        if Wallet.address_from_pubkey_hex(self.pubkey) != self.sender: return False
        msg_bytes = json.dumps(self.message(), separators=(',', ':'), sort_keys=True).encode()
        return Wallet.verify_signature(bytes.fromhex(self.signature), msg_bytes, self.pubkey)
    
    def is_timestamp_valid(self, current_time: Optional[float] = None) -> bool:
        if current_time is None: current_time = time.time()
        return (current_time - MAX_FUTURE_DRIFT) <= self.timestamp <= (current_time + MAX_FUTURE_DRIFT)

@dataclass
class CoinbaseTx:
    recipient: str
    amount: Decimal
    def txhash(self, hashfunc: Callable = hashlib.sha256) -> str:
        return hash_json({"type": "coinbase", "to": self.recipient, "amt": self.amount}, hashfunc)
    def to_dict(self) -> dict:
        return {"type": "coinbase", "recipient": self.recipient, "amount": str(self.amount)}

## 5. Block Structure

A container structure for validated transactions.

* **Block Header:** Contains metadata essential for Proof-of-Work and validity checks:
    * `merkle_root`: Ensures transaction data has not been tampered with.
    * `timestamp`: Prevents time-warp attacks.
    * `difficulty`: Records the network difficulty at the time of mining.
    * `nonce`: The value found by the miner to solve the PoW puzzle.

In [28]:
@dataclass
class Block:
    height: int
    timestamp: float
    transactions: List[Union[CoinbaseTx, AccountTx]]
    previous_hash: str
    merkle_root: str
    nonce: int = 0
    difficulty: int = 1
    
    def header_dict(self) -> dict:
        return {
            "height": self.height, "timestamp": self.timestamp,
            "previous_hash": self.previous_hash, "merkle_root": self.merkle_root,
            "nonce": self.nonce, "difficulty": self.difficulty
        }
    def blockhash(self, hashfunc: Callable = hashlib.sha256) -> str:
        return hash_json(self.header_dict(), hashfunc)

## 6. State and Mempool Managers

These modules manage the ledger state and enforce security rules.

* **StateManager (World State):** Maps addresses directly to balances and nonces. This design allows for $O(1)$ lookup complexity, simplifying state management compared to UTXO.
* **TransactionValidator (Adaptive Precision & Security):**
    * **Adaptive Precision:** Dynamically limits the allowed decimal places based on network difficulty ($D$). The smallest unit becomes $10^{-D}$. This prevents "dust spam" when difficulty is low and allows finer granularity as the network grows.
    * **Replay Protection:** Enforces that the transaction nonce exactly matches the account's expected nonce.
* **MempoolManager (DoS Mitigation):** A waiting area for unconfirmed transactions. It implements governance rules (such as `max_pending_size`) to reject spam and prevent memory exhaustion (DoS attacks).

In [ ]:
class StateManager:
    def __init__(self):
        self.balances: Dict[str, int] = {}
        self.nonces: Dict[str, int] = {}
    def get_balance(self, addr: str) -> int: return self.balances.get(addr, 0)
    def get_nonce(self, addr: str) -> int: return self.nonces.get(addr, 0)
    def update_balance(self, addr: str, amount: int): self.balances[addr] = self.balances.get(addr, 0) + amount
    def deduct_balance(self, addr: str, amount: int): self.balances[addr] = self.balances.get(addr, 0) - amount
    def increment_nonce(self, addr: str): self.nonces[addr] = self.nonces.get(addr, 0) + 1
    def copy_state(self): return self.balances.copy(), self.nonces.copy()

class TransactionValidator:
    def __init__(self, hashfunc: Callable):
        self.hashfunc = hashfunc
        
    def check_precision(self, amount: Decimal, difficulty: int) -> bool:
        s = str(amount)
        if '.' not in s: return True 
        decimals = len(s.split('.')[1])
        return decimals <= difficulty

    def validate_and_apply(self, tx: AccountTx, balances: Dict, nonces: Dict, current_difficulty: int = 1) -> bool:
        if tx.amount <= 0 or tx.fee < 0: return False
        if not tx.is_timestamp_valid(): return False
        if not tx.verify_signature(): return False
        
        if not self.check_precision(tx.amount, current_difficulty): return False
        
        curr_nonce = nonces.get(tx.sender, 0)
        if tx.nonce != curr_nonce: return False
        
        req = tx.amount + tx.fee
        if balances.get(tx.sender, Decimal("0")) < req: return False
        
        balances[tx.sender] -= req
        balances[tx.recipient] = balances.get(tx.recipient, Decimal("0")) + tx.amount
        nonces[tx.sender] = curr_nonce + 1
        return True

@dataclass
class MempoolConfig:
    max_pending_size: int = 10000
    max_per_sender: int = 100
    min_fee: int = 1
    max_wait_blocks: int = 100

class MempoolManager:
    def __init__(self, config: MempoolConfig, hashfunc: Callable):
        self.config = config
        self.hashfunc = hashfunc
        self.pending: List[AccountTx] = []
        self.added_height: Dict[str, int] = {}

    def add_transaction(self, tx: AccountTx, current_height: int) -> bool:
        if len(self.pending) >= self.config.max_pending_size: return False
        self.pending.append(tx)
        self.added_height[tx.txhash(self.hashfunc)] = current_height
        return True

    def expire_old_transactions(self, current_height: int):
        self.pending = [tx for tx in self.pending 
                        if current_height - self.added_height.get(tx.txhash(self.hashfunc), 0) <= self.config.max_wait_blocks]

    def remove_transactions(self, tx_hashes: set):
        self.pending = [tx for tx in self.pending if tx.txhash(self.hashfunc) not in tx_hashes]

## 7. Consensus Engine (PoWChain Core Logic)

Orchestrates the Proof-of-Work consensus algorithm and blockchain maintenance.

* **Mining & Fee Market:** Miners select transactions from the mempool, sorting them by fee density to maximize profit (Fee Market).
* **Dynamic Difficulty Adjustment:** Maintains a stable block generation time (target: ~1.0s) by adjusting the mining difficulty every 5 blocks.
    * If blocks are mined too fast, difficulty increases (+1); if too slow, it decreases (-1).
* **Chain Reorganization:** Implements logic to resolve forks. If a valid chain with more accumulated work is received, the node switches to it to ensure consensus.

In [ ]:
class PoWChain:
    def __init__(self, difficulty=4, block_reward=Decimal("50"), hashfunc=hashlib.sha256):
        self.initial_difficulty = difficulty
        self.block_reward = block_reward
        self.hashfunc = hashfunc
        self.block_capacity = 1000
        
        self.state = StateManager()
        self.mempool = MempoolManager(MempoolConfig(), hashfunc)
        self.validator = TransactionValidator(hashfunc)
        
        self.chain = []
        self._create_genesis_block()

    def _create_genesis_block(self):
        genesis = Block(height=0, timestamp=time.time(), transactions=[], previous_hash="0"*64, merkle_root=merkle_root([], self.hashfunc), nonce=0, difficulty=self.initial_difficulty)
        self.chain.append(genesis)

    def validate_block(self, block: Block, previous_block: Block) -> bool:
        if block.previous_hash != previous_block.blockhash(self.hashfunc): return False
        if not block.blockhash(self.hashfunc).startswith("0" * block.difficulty): return False
        if block.timestamp > time.time() + MAX_FUTURE_DRIFT: return False
        if block.timestamp <= previous_block.timestamp: return False
        return True

    def _get_next_difficulty(self) -> int:
        ADJUSTMENT_INTERVAL = 5
        TARGET_TIME = 1.0 
        current_height = len(self.chain)
        last_block = self.chain[-1]
        
        if current_height == 0 or current_height % ADJUSTMENT_INTERVAL != 0: return last_block.difficulty
        
        prev_block = self.chain[current_height - ADJUSTMENT_INTERVAL]
        actual_time = last_block.timestamp - prev_block.timestamp
        
        if actual_time < TARGET_TIME * ADJUSTMENT_INTERVAL / 2: return last_block.difficulty + 1
        elif actual_time > TARGET_TIME * ADJUSTMENT_INTERVAL * 2: return max(1, last_block.difficulty - 1)
        return last_block.difficulty

    def mine_block(self, miner_wallet) -> Block:
        current_height = len(self.chain)
        self.mempool.expire_old_transactions(current_height)
        
        next_diff = self._get_next_difficulty()
        
        temp_bal, temp_nonce = self.state.copy_state()
        selected_txs = []
        total_fees = Decimal("0")
        
        candidates = sorted(self.mempool.pending, key=lambda x: x.fee, reverse=True)
        for tx in candidates:
            if len(selected_txs) >= self.block_capacity: break
            
            if self.validator.validate_and_apply(tx, temp_bal, temp_nonce, next_diff):
                selected_txs.append(tx)
                total_fees += tx.fee
        
        coinbase = CoinbaseTx(miner_wallet.address, self.block_reward + total_fees)
        all_txs = [coinbase] + selected_txs
        
        new_timestamp = time.time()
        if new_timestamp <= self.chain[-1].timestamp: new_timestamp = self.chain[-1].timestamp + 0.001
            
        block = Block(height=current_height, timestamp=new_timestamp, transactions=all_txs, previous_hash=self.chain[-1].blockhash(self.hashfunc), merkle_root=merkle_root([t.txhash(self.hashfunc) for t in all_txs], self.hashfunc), nonce=0, difficulty=next_diff)

        target = "0" * next_diff
        while not block.blockhash(self.hashfunc).startswith(target): block.nonce += 1
            
        for tx in selected_txs:
            self.state.deduct_balance(tx.sender, tx.amount + tx.fee)
            self.state.update_balance(tx.recipient, tx.amount)
            self.state.increment_nonce(tx.sender)
        self.state.update_balance(miner_wallet.address, coinbase.amount)
        self.mempool.remove_transactions({t.txhash(self.hashfunc) for t in selected_txs})
        self.chain.append(block)
        return block

    def add_transaction(self, tx):
        # 1. State/Balance Check
        if self.state.get_balance(tx.sender) < (tx.amount + tx.fee):
            return False
        if self.state.get_nonce(tx.sender) != tx.nonce:
            return False

        current_difficulty = self.chain[-1].difficulty
        if not self.validator.check_precision(tx.amount, current_difficulty):
            print(f"  [Reject] Precision Error: Decimals exceed current difficulty ({current_difficulty})")
            return False

        # 2. Mempool Conflict Check 
        for pending_tx in self.mempool.pending:
            if pending_tx.sender == tx.sender and pending_tx.nonce == tx.nonce:
                print(f"  [Reject] Double Spend detected: {tx.nonce}")
                return False

        # 3. Add to Mempool
        return self.mempool.add_transaction(tx, len(self.chain))

    def attempt_reorg(self, candidate_chain: List[Block]) -> bool:
        if len(candidate_chain) <= len(self.chain): return False
        temp_state = StateManager()
        for i, block in enumerate(candidate_chain):
            if i == 0:
                if block.previous_hash != "0"*64: return False
                continue
            if not self.validate_block(block, candidate_chain[i-1]): return False
            coinbase = block.transactions[0]
            fees = Decimal("0")
            for tx in block.transactions[1:]:
                if not self.validator.validate_and_apply(tx, temp_state.balances, temp_state.nonces, block.difficulty): return False
                fees += tx.fee
            if coinbase.amount != self.block_reward + fees: return False
            temp_state.update_balance(coinbase.recipient, coinbase.amount)
        self.chain = copy.deepcopy(candidate_chain)
        self.state = temp_state
        return True

## 8. P2P Node Simulation

Simulates distributed node behavior and the **Longest Chain Rule**.

* **Syncing:** In a decentralized network, nodes must agree on a single history. If a peer presents a valid chain that is longer than the local chain, the node accepts it as the truth. This mechanism resolves network partitions and prevents double-spending in the long run.

In [31]:
class Node:
    def __init__(self, name):
        self.name = name
        self.blockchain = PoWChain(difficulty=3) 
    
    def mine(self, miner_wallet):
        print(f"[{self.name}] Mining...")
        blk = self.blockchain.mine_block(miner_wallet)
        print(f"[{self.name}] Mined Block #{blk.height} (Diff: {blk.difficulty})")
        return blk
        
    def receive_chain(self, peer):
        if len(peer.blockchain.chain) > len(self.blockchain.chain):
            print(f"[{self.name}] Found longer chain! Syncing... ({len(self.blockchain.chain)} -> {len(peer.blockchain.chain)})")
            self.blockchain = copy.deepcopy(peer.blockchain)
            return True
        return False

## 9. Simulation and Security Analysis

Validates the theoretical architecture through specific scenarios described in the report:

1.  **Standard Operations:** Verifies the lifecycle of transactions and the Fee Market mechanism.
2.  **Double Spending Prevention:** Checks if the system correctly detects and rejects a second transaction attempting to spend the same nonce (Mempool Collision).
3.  **Fork Resolution:** Simulates a network partition and confirms that nodes converge on the longest chain.
4.  **Security Analysis:**
    * **Replay Attack:** Confirms that re-broadcasting an old transaction fails due to nonce mismatch.
    * **DoS Mitigation:** Tests the mempool's resilience against flooding attacks by enforcing capacity limits.

In [32]:
# --- FINAL SIMULATION ---
print("=== 1. Currency Generation & Transactions ===")
node_A = Node("Node A")
alice = Wallet()
bob = Wallet()
miner = Wallet()

# Miner gets coins via Coinbase
node_A.mine(miner) 
print(f"Miner Balance: {node_A.blockchain.state.get_balance(miner.address)}")

# Miner sends to Alice
tx1 = AccountTx(miner.address, alice.address, 50, fee=1, nonce=0)
tx1.sign(miner)
node_A.blockchain.add_transaction(tx1)
node_A.mine(miner)
print(f"Alice Balance: {node_A.blockchain.state.get_balance(alice.address)}")

print("\n=== 2. Double Spending Prevention ===")
# Alice tries to double spend
tx_valid = AccountTx(alice.address, bob.address, 50, fee=1, nonce=0)
tx_valid.sign(alice)

tx_double = AccountTx(alice.address, miner.address, 50, fee=1, nonce=0)
tx_double.sign(alice)

print("Sending TX 1 (Valid):")
node_A.blockchain.add_transaction(tx_valid) # OK

print("Sending TX 2 (Double Spend):")
node_A.blockchain.add_transaction(tx_double) # Reject

print("\n=== 3. P2P Consensus & Fork Resolution ===")
node_B = Node("Node B")
# Create Fork
node_A.mine(miner)
node_B.mine(miner)

print(f"Node A Height: {len(node_A.blockchain.chain)}")
print(f"Node B Height: {len(node_B.blockchain.chain)}")

# A mines one more
node_A.mine(miner)
print("Node A mined another block.")

# B syncs with A
node_B.receive_chain(node_A)
print(f"Node B Height after sync: {len(node_B.blockchain.chain)}")

=== 1. Currency Generation & Transactions ===
[Node A] Mining...
[Node A] Mined Block #1 (Diff: 3)
Miner Balance: 50
[Node A] Mining...
[Node A] Mined Block #2 (Diff: 3)
Alice Balance: 0

=== 2. Double Spending Prevention ===
Sending TX 1 (Valid):
Sending TX 2 (Double Spend):

=== 3. P2P Consensus & Fork Resolution ===
[Node A] Mining...
[Node A] Mined Block #3 (Diff: 3)
[Node B] Mining...
[Node B] Mined Block #1 (Diff: 3)
Node A Height: 4
Node B Height: 2
[Node A] Mining...
[Node A] Mined Block #4 (Diff: 3)
Node A mined another block.
[Node B] Found longer chain! Syncing... (2 -> 5)
Node B Height after sync: 5


In [38]:
# ==========================================
# FINAL SIMULATION: SECURITY ANALYSIS
# ==========================================

# Initialize clean test environment
sec_node = Node("SecurityNode")
alice = Wallet(); bob = Wallet(); attacker = Wallet(); miner = Wallet()

# Fund Alice with 100 Coins
sec_node.blockchain.state.update_balance(alice.address, Decimal("100"))
print(f"Alice Initial Balance: {sec_node.blockchain.state.get_balance(alice.address)}")

print("\n=== 1. Replay Attack Verification ===")
# [Scenario] Alice -> Bob (Nonce 0). Attacker broadcasts the identical signature again.

# 1) Valid Transaction
tx_valid = AccountTx(alice.address, bob.address, Decimal("10"), fee=Decimal("1"), nonce=0)
tx_valid.sign(alice)

if sec_node.blockchain.add_transaction(tx_valid):
    print("Step 1: Original Tx accepted.")
    sec_node.mine(miner) # Mined -> Alice's State Nonce is now 1
else:
    print("Step 1 FAIL.")

# 2) Replay Attack (Nonce is still 0)
print("Step 2: Attacker broadcasts the SAME transaction again...")
if sec_node.blockchain.add_transaction(tx_valid):
    print(" FAIL: Replay Attack Succeeded (Vulnerability!).")
else:
    print(" SUCCESS: Replay Attack Rejected (Nonce mismatch).")


print("\n=== 2. Double Spending Verification ===")
# [Scenario] Alice's Nonce is 1. Sign and broadcast to different recipients simultaneously.

tx_A = AccountTx(alice.address, bob.address, Decimal("50"), fee=Decimal("1"), nonce=1)
tx_B = AccountTx(alice.address, attacker.address, Decimal("50"), fee=Decimal("1"), nonce=1)
tx_A.sign(alice)
tx_B.sign(alice)

print("Broadcasting Tx A (Alice -> Bob)...")
if sec_node.blockchain.add_transaction(tx_A):
    print(" -> Tx A accepted.")
else:
    print(" -> Tx A rejected.")

print("Broadcasting Tx B (Same Nonce)...")
# Should be rejected due to Mempool Collision check (PoWChain's add_transaction)
if sec_node.blockchain.add_transaction(tx_B):
    print(" FAIL: Double Spend Accepted (Vulnerability!).")
else:
    print(" SUCCESS: Double Spend Rejected (Mempool Collision).")


print("\n=== 3. DoS (Mempool Flooding) Verification ===")

# [Setup] 1. Initialize Mempool and set capacity limit
sec_node.blockchain.mempool.pending = []
sec_node.blockchain.mempool.config.max_pending_size = 5
print(f"Mempool Capacity Limited to: {sec_node.blockchain.mempool.config.max_pending_size}")

# [Setup] 2. Create 10 attacker wallets and fund them
# (Using different wallets to avoid Nonce chaining issues)
attackers = [Wallet() for _ in range(10)]
for atk in attackers:
    # Fund enough for fees (Direct State manipulation)
    sec_node.blockchain.state.update_balance(atk.address, Decimal("10"))

print("Attacker bots created. Starting flood...")

success_count = 0
for i, atk in enumerate(attackers):
    # Each attacker sends Tx with Nonce 0 (No conflict since State Nonce is 0)
    tx = AccountTx(atk.address, bob.address, Decimal("0.1"), fee=Decimal("0.1"), nonce=0)
    tx.sign(atk)
    
    if sec_node.blockchain.add_transaction(tx):
        success_count += 1
        print(f" -> Tx #{i+1} accepted (Mempool Size: {len(sec_node.blockchain.mempool.pending)})")
    else:
        print(f" -> Tx #{i+1} rejected (Mempool Full).")

if success_count == 5:
    print(f" SUCCESS: DoS Protection Working. Accepted {success_count}/10 transactions.")
else:
    print(f" FAIL: Mempool accepted {success_count} transactions (Expected 5).")

Alice Initial Balance: 100

=== 1. Replay Attack Verification ===
Step 1: Original Tx accepted.
[SecurityNode] Mining...
[SecurityNode] Mined Block #1 (Diff: 3)
Step 2: Attacker broadcasts the SAME transaction again...
 SUCCESS: Replay Attack Rejected (Nonce mismatch).

=== 2. Double Spending Verification ===
Broadcasting Tx A (Alice -> Bob)...
 -> Tx A accepted.
Broadcasting Tx B (Same Nonce)...
  [Reject] Double Spend detected: 1
 SUCCESS: Double Spend Rejected (Mempool Collision).

=== 3. DoS (Mempool Flooding) Verification ===
Mempool Capacity Limited to: 5
Attacker bots created. Starting flood...
 -> Tx #1 accepted (Mempool Size: 1)
 -> Tx #2 accepted (Mempool Size: 2)
 -> Tx #3 accepted (Mempool Size: 3)
 -> Tx #4 accepted (Mempool Size: 4)
 -> Tx #5 accepted (Mempool Size: 5)
 -> Tx #6 rejected (Mempool Full).
 -> Tx #7 rejected (Mempool Full).
 -> Tx #8 rejected (Mempool Full).
 -> Tx #9 rejected (Mempool Full).
 -> Tx #10 rejected (Mempool Full).
 SUCCESS: DoS Protection Worki